> **Disclaimer**  
> This notebook uses content adapted from a course-provided notebook available at [github.com/langchain-ai/lca-langchainV1-essentials](https://github.com/langchain-ai/lca-langchainV1-essentials).  
> All original credit belongs to the course authors.

# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

Load and/or check for needed environmental variables

In [6]:
from dotenv import load_dotenv
load_dotenv()

from bedrock import nova_pro as llm

In [7]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [8]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [9]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [10]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## Build a Dynamic Prompt
Utilize runtime context and middleware to generate a dynamic prompt.

In [11]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [13]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [16]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

[{'type': 'text', 'text': '<thinking> To find the most costly purchase by Frank Harris, I need to identify the tracks associated with Frank Harris and then determine the maximum cost among those tracks. First, I need to find the TrackId of tracks by Frank Harris. Then, I can use this TrackId to find the maximum cost in the Track table. </thinking>\n'}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': "SELECT TrackId FROM Track WHERE Composer = 'Frank Harris'"}, 'id': 'tooluse__PPYGz2BR_yfjIjxyrO-sg'}]
Tool Calls:
  execute_sql (tooluse__PPYGz2BR_yfjIjxyrO-sg)
 Call ID: tooluse__PPYGz2BR_yfjIjxyrO-sg
  Args:
    query: SELECT TrackId FROM Track WHERE Composer = 'Frank Harris'
================================= Tool Message =================================
Name: e

In [17]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking> To determine the most costly purchase by Frank Harris, I need to find the purchase with the highest amount associated with him. I will first identify the table that likely contains purchase data, which could be named something like 'purchases' or 'transactions'. I will then look for columns that might contain the customer's name, such as 'customer_name', and the purchase amount, such as 'amount'. Once I have this information, I can query the database to find the maximum amount for Frank Harris. </thinking>\n"}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': 'SELECT DISTINCT customer_name FROM purchases LIMIT 5'}, 'id': 'tooluse_7PDLDoOSSfKW2K27WVdsvQ'}]
Tool Calls:
  execute_sql (tooluse_7PDLDoOSSfKW2K27WVdsvQ)
 Call ID: t